In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [3]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [4]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
import random
import math

In [5]:
def load_dataset():
  train_data = pd.read_csv('trainset.csv')
  y_train = pd.read_csv('delta_theta.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [ ]:
# train_data.head()
# y_train.head()

In [6]:
train_data, y_train, target_data = load_dataset()

In [ ]:
# train_data.shape
# target_data.shape

In [7]:
num_classes = 4
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [8]:
train_data.shape
target_data.shape

(333, 128, 128)

In [9]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(333, 128, 128, 3)

In [10]:
target_data = rgb_target

In [11]:
target_data.shape

(333, 128, 128, 3)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [13]:
input_shape = (128, 128, 3)

In [ ]:
# print(y_test)

In [18]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [19]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [20]:
conv = concatenate([current_model, target_model])

In [33]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=None)(dense)
dense = Dropout(0.5)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='linear')(dense)
# output = math.floor(output)
model = Model(inputs=[current_input, target_input], outputs=[output])

opt = Adam(learning_rate=0.0001)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])


In [37]:
best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_mse', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [38]:
model.fit([x_train, target_xtrain], y_train,
          epochs=100,
          validation_split=0.3,
          callbacks = callbacks,
          verbose=1)

Epoch 1/100
6/6 [==============================] - 1s 121ms/step - loss: 136.3442 - mse: 136.3442 - val_loss: 189.2971 - val_mse: 189.2971

Epoch 00001: val_mse improved from -inf to 189.29710, saving model to weights.best.hdf5
Epoch 2/100
6/6 [==============================] - 1s 124ms/step - loss: 124.8706 - mse: 124.8706 - val_loss: 190.8282 - val_mse: 190.8282

Epoch 00002: val_mse improved from 189.29710 to 190.82817, saving model to weights.best.hdf5
Epoch 3/100
6/6 [==============================] - 1s 124ms/step - loss: 112.2422 - mse: 112.2422 - val_loss: 186.3265 - val_mse: 186.3265

Epoch 00003: val_mse did not improve from 190.82817
Epoch 4/100
6/6 [==============================] - 1s 123ms/step - loss: 104.6768 - mse: 104.6768 - val_loss: 182.1214 - val_mse: 182.1214

Epoch 00004: val_mse did not improve from 190.82817
Epoch 5/100
6/6 [==============================] - 1s 117ms/step - loss: 96.1294 - mse: 96.1294 - val_loss: 183.5373 - val_mse: 183.5373

Epoch 00005: val_

In [39]:
model.evaluate([x_test, target_xtest], y_test)

4/4 [==============================] - 0s 32ms/step - loss: 255.0922 - mse: 255.0922


[255.0922088623047, 255.0922088623047]

In [40]:
prediction = model.predict([x_test, target_xtest])
prediction = np.rint(prediction)

In [41]:
print(prediction)

[[  7.  -8.  -5.   2.]
 [ 33.  -4.  -5.   5.]
 [ -6.  -3.  -5.   4.]
 [ -7.  -4.  -3.   4.]
 [ 40.  -4.  -3.   7.]
 [ -2.   2.  -8.   1.]
 [ 13.  -4.   1.   5.]
 [  2.  -2.  -6.   5.]
 [ 37.  -8.   4.   4.]
 [ 39.   5. -10.   7.]
 [  0.  -2.  -9.   4.]
 [ 17.  -3.  -5.  -0.]
 [ 23.  -0.  -3.  10.]
 [ -7.  -2.  -3.   3.]
 [ 26.   8. -14.   9.]
 [ 15.  -9.  -0.   3.]
 [ 15.   2. -11.   5.]
 [ 20.   4. -12.   4.]
 [ 21.   7.  -8.   8.]
 [ -6.   2. -12.  10.]
 [  6.   2. -11.   1.]
 [ 10.  -4.  -1.   4.]
 [  5.  -1.  -3.  14.]
 [-12. -10.  -7.   7.]
 [ -0.   8. -11.   5.]
 [ 41.   1.  -5.   4.]
 [ 29.   6. -10.   9.]
 [ 38.   4.  -6.   7.]
 [ 47.  -4.  -3.   5.]
 [  9.   3.  -8.   6.]
 [ 32.  -9.   0.  -2.]
 [ 16.   5. -14.   2.]
 [ 18. -11.   1.   4.]
 [ 11.   2.  -8.   0.]
 [ 49.  -5.  -0.   1.]
 [ 41.  -7.  -0.   7.]
 [ 40.  -9.  -4.   8.]
 [ 31.  -1.  -9.   6.]
 [ 32.   0. -11.   3.]
 [ 43.  -0.  -2.   3.]
 [ 39.  -2.  -4.   5.]
 [ 27. -11.   2.  12.]
 [ 54.   1. -10.   3.]
 [ -1.   6.

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg